## Imports

In [1]:
import torch
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
import matplotlib.pyplot as plt
import os
import warnings

warnings.simplefilter("ignore", category=UserWarning)

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)

mps


## Environment

In [15]:
import ale_py

env = gym.make("ALE/Pong-v5", render_mode = "human")

In [16]:
env.reset()

(array([[[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [109, 118,  43],
         [109, 118,  43],
         [109, 118,  43]],
 
        [[109, 118,  43],
         [109, 118,  43],
         [109, 118,  43],
         ...,
         [109, 118,  43],
         [109, 118,  43],
         [109, 118,  43]],
 
        [[109, 118,  43],
         [109, 118,  43],
         [109, 118,  43],
         ...,
         [109, 118,  43],
         [109, 118,  43],
         [109, 118,  43]],
 
        ...,
 
        [[ 53,  95,  24],
         [ 53,  95,  24],
         [ 53,  95,  24],
         ...,
         [ 53,  95,  24],
         [ 53,  95,  24],
         [ 53,  95,  24]],
 
        [[ 53,  95,  24],
         [ 53,  95,  24],
         [ 53,  95,  24],
         ...,
         [ 53,  95,  24],
         [ 53,  95,  24],
         [ 53,  95,  24]],
 
        [[ 53,  95,  24],
         [ 53,  95,  24],
         [ 53,  95,  24],
         ...,
         [ 53,  95,  24],
  

In [17]:
env.action_space

Discrete(6)

In [18]:
env.observation_space

Box(0, 255, (210, 160, 3), uint8)

In [19]:
env.render()

In [ ]:
episodes = 5
for i in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        obs, obs2, reward, done, info = env.step(action)
        score += reward
    print(f"Episode: {i} Score: {score}")

env.close()

## Train Model

In [21]:
env = DummyVecEnv([lambda: env])

In [22]:
log_path = os.path.join("Training", "logs")
model = PPO("CnnPolicy", env, verbose=1, tensorboard_log = log_path, device=device)

Using mps device
Wrapping the env in a VecTransposeImage.


In [23]:
model.learn(total_timesteps=100000)

Logging to Training/logs/PPO_1
-----------------------------
| time/              |      |
|    fps             | 14   |
|    iterations      | 1    |
|    time_elapsed    | 145  |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 14          |
|    iterations           | 2           |
|    time_elapsed         | 279         |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009364238 |
|    clip_fraction        | 0.0871      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.78       |
|    explained_variance   | -0.0111     |
|    learning_rate        | 0.0003      |
|    loss                 | -0.000978   |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.00895    |
|    value_loss           | 0.45        |
-----------------------------------------
---

## Save Model

In [24]:
ppo_path = os.path.join("Training", "Saved Models", "PPO_Driving_Model")

In [25]:
model.save(ppo_path)

In [26]:
del model

In [27]:
model = PPO.load(ppo_path, env)

Wrapping the env in a VecTransposeImage.


Use "tensorboard --logdir logs/" in the Training terminal to get to tensorboard

## Test Model

In [28]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(-13.8, 3.249615361854384)

In [29]:
episodes = 5

ep = []
points = []

for i in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward        

    ep.append(i)
    points.append(score)
    
    print(f"Episode: {i} Score: {score}")

env.close()

Episode: 1 Score: [-6.]
Episode: 2 Score: [-13.]
Episode: 3 Score: [-17.]
Episode: 4 Score: [-11.]
Episode: 5 Score: [-15.]
